In [1]:
!git clone https://gist.github.com/3c36fa706f8c1f982b7507d423e299f7.git

fatal: destination path '3c36fa706f8c1f982b7507d423e299f7' already exists and is not an empty directory.


In [2]:
import pandas

headlines_df = pandas.read_csv('3c36fa706f8c1f982b7507d423e299f7/3000_stock_headlines.csv')
headlines_df.head(5)


,Unnamed: 0.1,Unnamed: 0,headline,url,publisher,date,stock
0,1304084,1307105,Notable earnings after Thursday's close,http://seekingalpha.com/news/3225116-notable-e...,Seeking Alpha,2016-11-16 00:00:00,POST
1,871819,873877,"Intersil Beats 1Q Earnings, Revs Lag - Analyst...",http://www.zacks.com/stock/news/98175/intersil...,Zacks,2013-04-29 00:00:00,ISIL
2,171302,171756,Brunswick (BC) Plans To Spin-Off Fitness Busin...,https://seekingalpha.com/article/4153223-bruns...,Seeking Alpha,2018-03-05 00:00:00,BC
3,1645188,1649033,Consolidated Communications (CNSL) Q2 Earnings...,http://www.zacks.com/stock/news/453543/consoli...,Zacks,2019-07-30 00:00:00,TU
4,1161702,1164433,"Day Two Keynote Speaker, Charlie Riedl with CL...",http://www.gurufocus.com/news/1027489/day-two-...,GuruFocus,2020-02-03 00:00:00,NOG


In [3]:
import numpy as np

headlines_array = np.array(headlines_df)
headlines_list = list(headlines_array[:,2])
stocks_list = list(headlines_array[:, -1])

print(headlines_list)
print(stocks_list)

["Notable earnings after Thursday's close", 'Intersil Beats 1Q Earnings, Revs Lag - Analyst Blog', 'Brunswick (BC) Plans To Spin-Off Fitness Business - Slideshow', "Consolidated Communications (CNSL) Q2 Earnings: What's Up?", 'Day Two Keynote Speaker, Charlie Riedl with CLNG, Announced for EnerCom Dallas Energy ...', 'EQT Rubbishes JANA Claims on Rice Energy Buyout Synergies', 'Retailers look for government help for shipping crisis', 'Sempra Energy Donates To Fisher House Foundation As Part Of Commitment To Veteran Causes', 'Still Bullish on BioMarin, Lexicon Pharma', 'MoneyGram Shares Jump 50% As Western Union Reportedly Looks For Acquisition', "Domino's And Papa John's Face Major Threat As Pizza Industry Is Set To Change", 'GameStop near Support', 'Apple: A Top Fidelity Manager Sticks By The Stock', 'Ex-Dividend Stocks: Intel, Citigroup', 'Border tax seen as a profit headwind for major retailers', 'REITs: Something to Build On - Zacks Analyst Interviews', "What's Next For Oil?", 'Par

# Getting the tokenizer and the model


In [4]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")


# Installing Weights & Biases Python client and initializing a W&B Table

In [5]:
import wandb

wandb.init(project="FinBERT_Stock_Sentiment_Analysis")

headlines_table = wandb.Table(columns=["Headline", "Stock", "Positive", "Negative", "Neutral"])

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: harsshpandey23 (harsshpandey23-iiitdm-jabalpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# Performing inference on the stock market news headlines with the FinBERT model implemented in HuggingFace

In [6]:
import torch

def chunk_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


STRIDE = 100

model.eval()

n=0
for lines, stocks in zip(chunk_list(headlines_list, STRIDE), chunk_list(stocks_list, STRIDE)):
  
  input = tokenizer(lines, padding = True, truncation = True,  return_tensors='pt')
  
  outputs = model(**input)

  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)

  print(f"{n+1}/{int(len(headlines_list)/STRIDE)}") 

  for headline, stock, pos, neg, neutr in zip(lines,stocks, prediction[:, 0].tolist(), prediction[:, 1].tolist(), prediction[:, 2].tolist() ): 
    headlines_table.add_data(headline, stock, pos, neg, neutr)
   
  n+=1


1/30
2/30
3/30
4/30
5/30
6/30
7/30
8/30
9/30
10/30
11/30
12/30
13/30
14/30
15/30
16/30
17/30
18/30
19/30
20/30
21/30
22/30
23/30
24/30
25/30
26/30
27/30
28/30
29/30
30/30


\## Logging model predictions as W&B Table

In [7]:
wandb.run.log({"Financial Sentiment Analysis Table" : headlines_table})
wandb.run.finish()